<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#по-данным-habr_data-получить-таблицу-с-названиями-топ-3-статей-(по-rating)-для-каждого-автора-DataFrame:-[“author”,-“article_title”,-“position”-(1,-2,-3)]" data-toc-modified-id="по-данным-habr_data-получить-таблицу-с-названиями-топ-3-статей-(по-rating)-для-каждого-автора-DataFrame:-[“author”,-“article_title”,-“position”-(1,-2,-3)]-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>по данным habr_data получить таблицу с названиями топ-3 статей (по rating) для каждого автора DataFrame: [“author”, “article_title”, “position” (1, 2, 3)]</a></span></li><li><span><a href="#по-данным-habr_data-получить-топ-(по-встречаемости)-английских-слов-из-заголовков.-Возможное-решение:-1)-выделение-слов-с-помощью-регулярных-выражений,-2)-разделение-на-массивы-слов-3)-explode-массивовов-4)-группировка-с-подсчетом-встречаемости" data-toc-modified-id="по-данным-habr_data-получить-топ-(по-встречаемости)-английских-слов-из-заголовков.-Возможное-решение:-1)-выделение-слов-с-помощью-регулярных-выражений,-2)-разделение-на-массивы-слов-3)-explode-массивовов-4)-группировка-с-подсчетом-встречаемости-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>по данным habr_data получить топ (по встречаемости) английских слов из заголовков. Возможное решение: 1) выделение слов с помощью регулярных выражений, 2) разделение на массивы слов 3) explode массивовов 4) группировка с подсчетом встречаемости</a></span></li></ul></div>

In [88]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
spark = SparkSession.builder.appName('hm3').getOrCreate()

In [9]:
df_pyspark = spark.read.csv('C:\\Users\\sysn1\\OneDrive\\Рабочий стол\\SQL ssis\\habr_data.csv', header = True, inferSchema=True)

### по данным habr_data получить таблицу с названиями топ-3 статей (по rating) для каждого автора DataFrame: [“author”, “article_title”, “position” (1, 2, 3)]

In [134]:
habr = df_pyspark.select(
                        'author_name',
                        'title',
                        col('rating').cast(IntegerType())                   
                        )

habr = habr.dropna('any', subset='rating')

habr.printSchema()

root
 |-- author_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)



In [236]:
windowSpec = Window.partitionBy('author_name').orderBy(col('rating').desc())
top_title = habr.withColumn('top', row_number().over(windowSpec))

top_title = top_title.select('*').where('top < 4').cache()
top_title.show()

+-----------------+--------------------+------+---+
|      author_name|               title|rating|top|
+-----------------+--------------------+------+---+
|    Alik Chebotar|Массовая рассылка...|     3|  1|
|   Andrey Nevolin|Измерение времени...|    39|  1|
|   Eugene Voityuk|     Отстой ли XMPP?|    33|  1|
|      Glyph media|«Всем иногда хоче...|    15|  1|
|      HedgeInSmog|Подключение графи...|    23|  1|
|  Mikhail Kulikov|Настройка окружен...|    18|  1|
|  Nikolay Garmash|Коллекция инструм...|     0|  1|
| Yuliya Statsenko|Dynamics 365 &amp...|     6|  1|
|            mammy|Стивен Хокинг о р...|    14|  1|
|            mammy|Большинство топ-м...|     7|  2|
|        nomadmoon|Восстановление ба...|    22|  1|
|       outcoldman|         WPF Browser|    10|  1|
|Андрей Алексейчук|Битва SEC против ...|    21|  1|
|  Анна Артамонова|Безопасность в ин...|    20|  1|
|Балыбердин Андрей|Цифровое бессмерт...|    24|  1|
| Вадим Закондырин|Обзор новых возмо...|    45|  1|
| Вадим Зако

### по данным habr_data получить топ (по встречаемости) английских слов из заголовков. Возможное решение: 1) выделение слов с помощью регулярных выражений, 2) разделение на массивы слов 3) explode массивовов 4) группировка с подсчетом встречаемости

In [235]:
top_word = top_title.select('title')\
    .withColumn('check',split(col('title'),' '))\
    .select(col('title'), explode(col('check')).alias('explode') )\
    .withColumn('eng_word', regexp_extract(col('explode'), '[A-Za-z]+', 0) )\
    .where('eng_word != ""')\
    .groupBy('eng_word').count().orderBy(col('count').desc()).cache()

top_word.show(50)

+----------+-----+
|  eng_word|count|
+----------+-----+
|    Google|  166|
|         C|  101|
|   Android|   94|
|   Windows|   92|
|      Java|   81|
|       API|   72|
|       PHP|   69|
|         D|   66|
|    Python|   62|
|     Linux|   61|
| Microsoft|   60|
|      Data|   56|
|       iOS|   50|
|JavaScript|   49|
|        IT|   48|
|       SQL|   48|
|        vs|   48|
|  Telegram|   39|
|Kubernetes|   38|
|    Server|   38|
|        Go|   37|
|       Web|   36|
|     Apple|   35|
|     React|   34|
|   Science|   32|
|PostgreSQL|   32|
|       for|   30|
|         c|   30|
|    Chrome|   30|
|     Intel|   29|
|    Spring|   27|
|       NET|   26|
|    Meetup|   26|
|     Phone|   24|
| Framework|   24|
|      Mail|   23|
|   Angular|   22|
|         I|   22|
|      HTML|   21|
|      Core|   21|
|      Ruby|   21|
|     Azure|   20|
|    Oracle|   20|
|    Apache|   20|
|        MS|   20|
|       CSS|   20|
|    Docker|   19|
|     Cisco|   19|
|        JS|   19|
|    Zabbix|